In [29]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
pd.set_option('display.max_columns', None)
%matplotlib inline

In [30]:
attack_file = pd.read_csv('attacks.csv', encoding= "ISO-8859-1")
f = attack_file.copy()
orig_shape = attack_file.shape

In [31]:
f.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-18,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-18,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-18,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-18,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [32]:
#f[f['Unnamed: 22'].notna() == True]
#f[f['Unnamed: 23'].notna() == True]
# Going to drop these columns since they are practically empty and have no real meaning
#f.drop(columns=['Unnamed: 22','Unnamed: 23'], axis=1,inplace=True)

In [33]:
f.shape

(25723, 24)

In [34]:
# Clean up column names

strip_dict = {}
for i,j in enumerate(f.columns):
    strip_dict[j] = j.strip()

f.rename(columns= strip_dict,inplace = True)

In [35]:
# identify high NaN lines and replace useless column
nun_nan = []

for fila in f.itertuples():
    
    check=[]
    
    for e in fila:
        
        check.append(pd.isna(e))
        
    nun_nan.append(sum(check))
f['Unnamed: 22'] = nun_nan
f.rename(columns={'Unnamed: 22':'NaN Columns'},inplace=True)


In [36]:
#identify nulls
nullity = pd.DataFrame([f.notna().sum(), f.isnull().sum()], ['Not Nulls','Nulls']).T
nullity.sort_values(by='Nulls',inplace=True)
nullity

,Not Nulls,Nulls
NaN Columns,25723,0
Case Number,8702,17021
original order,6309,19414
Date,6302,19421
Case Number.2,6302,19421
Case Number.1,6302,19421
href,6302,19421
pdf,6302,19421
href formula,6301,19422
Year,6300,19423


In [37]:
# eliminate NaN >10 rows
f.drop(index = f[f['NaN Columns']>10].index, inplace=True)
f.shape

(6302, 24)

In [38]:
# Identify strange years
f[['Year','Date','Time','Case Number']][f['Year'].isna()]
f[['Year','Date','Time','Case Number']][f['Year']<1500]

,Year,Date,Time,Case Number
6174,500.0,Circa 500 A.D.,NaN,0500.00.00
6175,77.0,77 A.D.,NaN,0077.00.00
6176,5.0,Ca. 5 A.D.,NaN,0005.00.00
6177,0.0,Ca. 214 B.C.,NaN,0.0214
6178,0.0,Ca. 336.B.C..,NaN,0.0336
...,...,...,...,...
6297,0.0,Before 1903,NaN,ND.0005
6298,0.0,Before 1903,NaN,ND.0004
6299,0.0,1900-1905,NaN,ND.0003
6300,0.0,1883-1889,NaN,ND.0002


In [39]:
#replace NaNs with correct digit
f.at[187,'Year'] = 2017
f.at[6079,'Year'] = 1836
print(f.iloc[187]['Year'],f.iloc[6079]['Year'])

#remove Non-well-defined years and unreliable
f.drop(index = f[f['Year']<1500].index,inplace = True)
print(f[['Year','Date','Time','Case Number']][(f['Year']<1500) | f['Year'].isna()])

2017.0 1836.0
Empty DataFrame
Columns: [Year, Date, Time, Case Number]
Index: []


In [40]:
#Explore Case Number
f[f['Case Number'].isna()] # 1 Nan that can be fixed
bad_index = f.index[f['Case Number'].isna()][0]
f.at[bad_index,'Case Number'] = '1905.09.05.R'

# find bad values
non_standard = []

for fila in f['Case Number']:
    try:
        nothing = re.findall('\d{4}.\d{2}.\d{2}',fila)[0]
    except:
        non_standard.append(fila,)

len(non_standard) # only 1 value

# correct bad case number index
bad_index = f['Case Number'].index[f['Case Number'] == non_standard[0]][0]
f.at[ bad_index , 'Case Number' ] = '1864.05.00'


In [41]:
# New date columns based on Case number
dates = []
for fila in f['Case Number']:
    dates.append(re.findall('(\d{4}).(\d{2}).(\d{2})?(.*)',fila))
new_years = []
new_months = []
new_days = []
extra = []

for date in dates:
    new_years.append(date[0][0])
    new_months.append(date[0][1])
    new_days.append(date[0][2])
    extra.append(date[0][3])


f['New Year'] = new_years
f['New Month'] = new_months
f['New Day'] = new_days
f['Notes'] = extra
f['DateConc'] = f['New Year'] + '-' + f['New Month'] + '-'+ f['New Day']



In [42]:
# Notes - Reporteds
f['Reported'] = np.where(f['Notes'].str.contains('R'),'Y','N')
f.shape

(6174, 30)

In [43]:
# Check for duplicates
f.duplicated().sum() == 0 # True, so now we can duplicate rows with several cases
f['Many'] = 0

g = pd.concat([f,f[f['Notes'] =='.a & b'],f[f['Notes'] =='.R.a & b']]) # Concatenate those which are duped within the row
g.reset_index(inplace=True,drop=True) #reset the index

# Count duplicates sequentially to identify separate cases
for i in range(len(g)):
    g.at[ i , 'Many' ] = g['DateConc'][(g['DateConc'].duplicated() == True) & (g.index <= i) & (g['DateConc'] == g['DateConc'][i])].count()+1



In [44]:
# find anomalies
g['New Month'].unique() # month = 94?
g['New Month'] = g['New Month'].replace('94','04')
g['New Day'].unique() # day = 38?
g['New Day'] = g['New Day'].replace('38','28')

In [45]:
# New case number is cleaner and identifies cases in same day
g['Case Number'] = g['New Year'] + '.' + g['New Month'] + '.' + g['New Day'] + ', ' + g['Many'].astype(str)


In [46]:

g.loc[(g['New Day'] != '00'),'Approx Day'] = g['New Day']
g.loc[(g['New Day'] == '00'),'Approx Day'] = '01'
g.loc[(g['New Month'] != '00'),'Approx Month'] = g['New Month']
g.loc[(g['New Month'] == '00'),'Approx Month'] = '01'
# Create new date Column
g['Date Approx'] = pd.to_datetime(g['Approx Day'] +'-'+g['Approx Month']+'-'+g['New Year'], infer_datetime_format=True,errors='coerce')
g.drop(g[g['Date Approx'].isna()].index,inplace=True) # Remove really low dates that pandas doesnt accept
g.shape


(6166, 34)

In [47]:
g[g['New Month'] == '00'].count() # 391 entries don't have a definite month and values range wildly, so will delete
g.drop(g[g['New Month'] == '00'].index,inplace=True)
g.shape
g[g['New Day'] == '00'].count() #346 entries but could still be useful

Case Number               346
Date                      346
Year                      346
Type                      346
Country                   339
Area                      294
Location                  280
Activity                  297
Name                      320
Sex                       278
Age                        88
Injury                    343
Fatal (Y/N)               313
Time                       48
Species                   168
Investigator or Source    345
pdf                       346
href formula              346
href                      346
Case Number.1             346
Case Number.2             346
original order            346
NaN Columns               346
Unnamed: 23                 0
New Year                  346
New Month                 346
New Day                   346
Notes                     346
DateConc                  346
Reported                  346
Many                      346
Approx Day                346
Approx Month              346
Date Appro

In [48]:
# Year column has no bad values
for i in g.Year:
    try:
        i = int(i)
    except:
        print(i)


In [49]:
# small test to check if years actually coincide

g['test column'] = 0
for i in range(len(g)):
    g['test column'] = int(g['New Year'].iloc[i]) - int(g['Year'].iloc[i])
g['test column'].unique() # only 0, so works, can now remove test column

# Change Date column to approximate Date column which contains the proper values with "reported" removed and added 1 to every unknown day
# Remove excess year/month/day columns



g.drop(columns=['test column','New Year','Date','Unnamed: 23', 'Approx Day','Approx Month','DateConc','Notes','New Day','New Month'], axis=1,inplace=True) 
g.rename(columns={'Date Approx':'Date'},inplace=True)

In [50]:
# Clean up the rest of the columns
for i in g.columns:
    try:
        g[i] = g[i].str.strip()
    except:
        continue


In [51]:
# Clean up Age column to get rid of uncertain ages

for i in range(len(g)):
    try:
        x = int(g['Age'].iloc[i])
        if x == 0:
            g.at[i, 'Age'] = np.nan
        else:
            g.at[i, 'Age'] = x
        
    except:
        g.at[i, 'Age'] = np.nan
g['Age'].isna().sum()


2740

In [52]:
# Clean up Sex column to get rid of uncertain sexes
g['Sex'].unique()
for i in g[(g['Sex'] != 'F') & (g['Sex'] != 'M')].index:

    g.at[i, 'Sex'] = np.nan


In [60]:
# clean up NaNs and empty rows

g.drop(g[g['Case Number'].isna()].index,inplace=True) 
g.fillna('Unknown',inplace=True)

In [76]:
g['Country'] = g['Country'].str.upper() # Standardize

g.drop(g[g['Country'].str.contains('/')].index,inplace=True) # Clean uncertain countries


USA                     2130
AUSTRALIA               1274
SOUTH AFRICA             544
NEW ZEALAND              119
BRAZIL                   112
                        ... 
ARGENTINA                  1
COMOROS                    1
GUATEMALA                  1
NETHERLANDS ANTILLES       1
GHANA                      1
Name: Country, Length: 173, dtype: int64

In [99]:
g['Type'].unique()
for i in g[(g['Type'] == 'Boat') | (g['Type'] == 'Boatomg')].index: # Standardize errors

    g.at[i, 'Type'] = 'Boating'

g.drop(g[g['Type'] == 'Questionable'].index,inplace=True) # Get rid of questionable data
g[g['Type'] == 'Invalid'] # 508 entries of questionable shark involvement, will keep for now to compare against sex data
g[g['Type'] == 'Sea Disaster']

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,NaN Columns,Reported,Many,Date
59,"2017.11.25, 1",2017.0,Sea Disaster,LIBYA,Unknown,Gars Garabulli,2 boats capsized,31 migrants,Unknown,Unknown,FATAL,Y,Unknown,Some drowned but other may have been killed by...,"TG Com 24, 11/25/2017",2017.11.25.R-Libya.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.11.25.R,2017.11.25.R,6244.0,6.0,Y,1.0,2017-11-25
465,"2014.12.28, 1",2014.0,Sea Disaster,GREECE,Unknown,33 nautical miles off Othonoi Island,Unknown,Passenger ferry Norman Atlantic,Unknown,Unknown,"Of 9 bodies recovered, one was bitten by a shark",N,Unknown,Shark involvement prior to death still to be d...,"Greek Reporter, 1/13/2015",2014.12.28.d-NormanAtlantic.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.12.28.d,2014.12.28.d,5838.0,7.0,N,1.0,2014-12-28
587,"2014.01.04, 1",2014.0,Sea Disaster,JAPAN,Okinawa Prefecture,Off Miyako Island,Sea disaster,Rianto,M,31,5 cm bite to left foot,N,Unknown,Unknown,"Focus Taiwan, 1/7/2014",2014.01.04-Rianto.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,pdf_directory2014.01.04-Riano.pdf,2014.01.04,2014.01.04,5716.0,4.0,N,1.0,2014-01-04
891,"2011.07.30, 1",2011.0,Sea Disaster,PHILIPPINES,Unknown,Unknown,Sea disaster,"Fishing vessel. Occupants Gerry Malabago, Mark...",M,29,The two Malabagos were bitten by sharks but su...,N,Unknown,Unknown,"ABS-CBN News, 8/17/2011",2011.07.30-Malabago.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2011.07.30,2011.07.30,5412.0,6.0,N,1.0,2011-07-30
1158,"2009.02.08, 1",2009.0,Sea Disaster,USA,Puerto Rico,Quebradillas,Air Disaster,occupant of a Cessna 206,M,Unknown,It is probable that all 5 passengers died on i...,Y,Unknown,Unknown,"C. Ekstander, GSAF",2009.02.08-PuertoRicoAirCrash.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2009.02.08,2009.02.08,5145.0,5.0,N,1.0,2009-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6125,"1806.04.24, 1",1896.0,Sea Disaster,USA,Unknown,Unknown,Unknown,Crew of the schooner Mary,M,Unknown,Shark scavenged on the dead sailors,Unknown,Unknown,Unknown,"Evening Post, 9/6/1806",1806.04.24-crew-of-the-Mary.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1806.04.24,1806.04.24,178.0,9.0,N,1.0,1806-04-24
6128,"1803.05.17, 1",1803.0,Sea Disaster,USA,South Carolina,Off Charleston,Unknown,Captain Jones,M,Unknown,No injury,N,Unknown,Unknown,"Evening Post, 6/13/1803",1803.05.17-Jones.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1803.05.17,1803.05.17,175.0,6.0,N,1.0,1803-05-17
6141,"1783.03.02, 1",1783.0,Sea Disaster,MARTINIQUE,Unknown,Foundering of the privateer Holker,Unknown,Unknown,M,Unknown,Many of the crew were taken by sharks,Y,06h00,Unknown,"C.Moore, GSAF",1783.03.02-Martinique.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1783.03.02,1783.03.02,162.0,7.0,N,1.0,1783-03-02
6174,"1959.07.03, 2",1959.0,Sea Disaster,CARIBBEAN SEA,PANAMA,"Off Cristobal, 200 miles northeast of the entr...",Columbian petrol barge Rio Atrato burned and sank,Teresea Britton (on raft) & a man (on floating...,Unknown,27,"FATAL X 2, 8 others missing. Survivors fought ...",Y,Unknown,Unknown,"Cambridge Daily News, 7/4/1959; V. M. Coppleso...",1959.07.03-Rio-Atranto.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1959.07.03.a & b,1959.07.03.a & b,2218.0,5.0,N,2.0,1959-07-03


In [62]:
g.to_csv('progress.csv',index=False)

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,NaN Columns,Reported,Many,Date
